# Now that we have our cleaned dataframes, let's do some classification

In [119]:
#Imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
covid_df = pd.read_csv('./data/clean_covid_data.csv')
gas_df = pd.read_csv('./data/gas_stations.csv')

In [10]:
covid_df.head(2)

,date,county,fips,place,confirmed_cases,note,x,y,log_cases,category,ordinal_cat
0,2020-10-23,Sonoma,97.0,Bodega,1,10 or fewer,-122.973889,38.345278,0.0,0,0
1,2020-07-02,Plumas,63.0,Indian Valley,1,NaN,-120.885787,40.081003,0.0,0,0


In [11]:
gas_df.head(2)

,name,longitude,latitude
0,Circle K & Minimart,-117.190777,34.057365
1,NaN,-117.225609,34.048575


In [ ]:
# train/test/split with just covid data (labeled dataset)

In [ ]:
X = covid_df[['x', 'y']]
y_cat = covid_df['ordinal_cat'] #for when we do our categorical classification
y_cts = covid_df['log_cases'] #for when we do regression based on 

# New Idea: do hand-done PCA

In [16]:
#just using bottom 5 as an example
df = covid_df.tail()
df

,date,county,fips,place,confirmed_cases,note,x,y,log_cases,category,ordinal_cat
1200,2020-10-25,Los Angeles,37.0,Long Beach,13013,NaN,-118.161014,33.805559,9.473704,4,4
1201,2020-10-23,Sacramento,67.0,Sacramento,14476,NaN,-121.494444,38.581667,9.580247,4,4
1202,2020-10-25,Santa Clara,85.0,San Jose,15817,NaN,-121.900000,37.333333,9.668841,4,4
1203,2020-10-23,Fresno,19.0,Fresno,17052,NaN,-119.766667,36.750000,9.744023,4,4
1204,2020-10-25,San Diego,73.0,San Diego,23644,NaN,-117.148962,32.692470,10.070865,4,4


In [49]:
def find_closest_points(target_point, df, num_points):
    #break target into x and y
    x_target = target[0]
    y_target = target[1]
    
    # for each row in the dataframe (can't really iterate)
    # first need to make these columns as lists so we can iterate
    df_xlist = df['x'].tolist()
    df_ylist = df['y'].tolist()
    
    distances = []
    for x in range(len(df_xlist)):
        
        #change in x and y for the stop and start coords
        delta_x = df_xlist[x] - x_target
        delta_y = df_ylist[x] - y_target
        #distance between start and stop
        d = np.sqrt((delta_x)**2 + (delta_y)**2)
        distances.append(d)
    
    #df['distances'] = distances # gave me a copy error
    new_df = df.copy()
    new_df['distances'] = distances
    new_df = new_df.sort_values(by = 'distances')
    
    # need to make sure we don't count the target point (get 1:4 instead of :3)
    return new_df[1:4]

In [87]:
# just choose a random point from the dataframe to visualize what's happening
target_x = float(covid_df.iloc[[-5]]['x']) 
target_y = float(covid_df.iloc[[-5]]['y'])
target = (target_x, target_y)

In [90]:
find_closest_points(target_point = target, df = covid_df, num_points = 3)['distances'].tolist()

[0.007691419959931177, 0.008555378717510472, 0.02343421790885981]

In [93]:
def get_alpha_for_target_point(target_point, df, num_points):
    
    # can't loop over length of dataframe, so I'll put the distances of the relevant points in a list
    distances = find_closest_points(target_point = target_point, df = df, num_points = num_points)['distances'].tolist()
    
    # and put their corresponding log_cases in a list
    log_cases = find_closest_points(target_point = target_point, df = df, num_points = num_points)['log_cases'].tolist()
    
    alpha = 0
    for x in range(len(distances)):
        if distances[x] != 0:
            alpha += log_cases[x]/distances[x]
        else:
            alpha +=0
    
    return alpha


In [94]:
get_alpha_for_target_point(target_point = target, df = covid_df, num_points = 3)

1358.9412730995564

In [95]:
# create column in dataframe of these alphas
covid_df_xlist = covid_df['x']
covid_df_ylist = covid_df['y']

alphas = []

n_neighbors = 3

for x in range(len(covid_df_xlist)):
    target = (covid_df_xlist[x], covid_df_ylist[x])
    alpha = get_alpha_for_target_point(target_point = target, df = covid_df, num_points = n_neighbors)
    alphas.append(alpha)
    
alphas

[84.2446036442287,
 44.51616212092759,
 86.90439832836414,
 148.81373530182626,
 52.84439889181487,
 6.727057359677954,
 226.39550280759843,
 65.52287190101812,
 1212.8816335490978,
 63.7535062121234,
 23.25443914314826,
 150.82839499038036,
 0.0,
 92.92913098485712,
 0.0,
 78.41254334867277,
 88.44516034525753,
 591.5280885415478,
 1263.3996554134214,
 103.84119228560166,
 120.684619693045,
 260.6769055873673,
 104.24398380440383,
 0.0,
 19.9634369051365,
 364.0012133210347,
 41.39509156848929,
 0.0,
 133.75754373397734,
 243.68288944030672,
 44.7698857817648,
 65.26801889642167,
 317.6130744339286,
 64.14556490364214,
 51.8716251937164,
 135.5390541609989,
 195.73519901054027,
 24.827597220726634,
 242.00103705040942,
 135.448397712372,
 34.63074807768131,
 0.0,
 438.5658590599876,
 77.05037410174596,
 243.68288944030672,
 20.998083947008517,
 4507.224895508925,
 49.115334606647615,
 1522.7392116236647,
 276.2827322937985,
 239.0874152450091,
 515.9117224826045,
 84.35578560494068,
 

In [99]:
covid_df['alpha'] = alphas

In [103]:
covid_df.tail()

,date,county,fips,place,confirmed_cases,note,x,y,log_cases,category,ordinal_cat,alpha
1200,2020-10-25,Los Angeles,37.0,Long Beach,13013,NaN,-118.161014,33.805559,9.473704,4,4,1358.941273
1201,2020-10-23,Sacramento,67.0,Sacramento,14476,NaN,-121.494444,38.581667,9.580247,4,4,270.148871
1202,2020-10-25,Santa Clara,85.0,San Jose,15817,NaN,-121.900000,37.333333,9.668841,4,4,242.710237
1203,2020-10-23,Fresno,19.0,Fresno,17052,NaN,-119.766667,36.750000,9.744023,4,4,145.433164
1204,2020-10-25,San Diego,73.0,San Diego,23644,NaN,-117.148962,32.692470,10.070865,4,4,382.697827


In [167]:
covid_df.to_csv('./data/covid_with_alpha.csv', index = False)

#### Now we can use our new alpha column

# Predicting the categorical variable ordinal_cat

In [184]:
X = covid_df[['x', 'y', 'alpha']] # unfortunately adding the alpha overfit it to the training data
y = covid_df['ordinal_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22)

## KNN

In [185]:
pipe = make_pipeline(StandardScaler(), KNeighborsClassifier())
#pipe = make_pipeline(KNeighborsClassifier())

params = {
    'kneighborsclassifier__n_neighbors': [1, 2, 3],#list(range(5, 25, 2)), #how many decision trees are under the hood
    'kneighborsclassifier__weights': ['uniform'],# 'distance']
}

grid = GridSearchCV(pipe, param_grid = params, cv = 5, n_jobs = -1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'kneighborsclassifier__n_neighbors': [1, 2, 3],
                         'kneighborsclassifier__weights': ['uniform']})

In [186]:
grid.score(X_train, y_train)

0.9867109634551495

In [187]:
grid.score(X_test, y_test)

0.3708609271523179

In [188]:
grid.best_params_

{'kneighborsclassifier__n_neighbors': 1,
 'kneighborsclassifier__weights': 'uniform'}

In [189]:
grid.predict_proba(X_test)

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])

## Random Forests

In [190]:
params = {
    'n_estimators': [100, 200, 500, 1000, 2000], #how many decision trees are under the hood
    'max_depth': [2, 5, 7, 9, 11],
    'min_samples_split': [1, 2, 3, 4],
    'min_samples_leaf': [1, 2, 3]
}
forest = RandomForestClassifier()
grid = GridSearchCV(forest, param_grid = params, cv = 5, n_jobs = -1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 5, 7, 9, 11],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [1, 2, 3, 4],
                         'n_estimators': [100, 200, 500, 1000, 2000]})

In [193]:
grid.score(X_train, y_train)

0.7187153931339978

In [191]:
grid.score(X_test, y_test)

0.3675496688741722

In [192]:
grid.best_params_

{'max_depth': 9,
 'min_samples_leaf': 3,
 'min_samples_split': 4,
 'n_estimators': 100}

# Predicting the continuous variable log_cases

In [118]:
X = covid_df[['x', 'y', 'alpha']]
y = covid_df['log_cases']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22)

In [ ]:
## Lasso

In [ ]:
pipe = make_pipeline(SelectKBest(f_regression), Lasso()) #NO, select k best does this for selectign number of features